# Validation Set 1: diffuPy + PathMe  

In [1]:
import os
dir_path = os.path.dirname(os.path.realpath('__file__'))

In [2]:
from openpyxl import load_workbook
from collections import defaultdict
import pybel
import pybel_tools as pbt
import networkx as nx

from pybel.dsl import Abundance, BiologicalProcess, CentralDogma, ListAbundance, Reaction

from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL, PATHME_DIR

In [3]:
pybel.get_version()

'0.13.1'

### Load Data Set 1: Input Scores

In [4]:
def munge_labels(label):
    """Process ene"""
    remove_set = ['*', ' ', '|', '-', '"', "'"]
    
    label = str(label).lower()
    
    for symb in remove_set:
        if symb in label:
            label = label.replace(symb, '')
    
    if '/' in label:
        label = tuple(set(label.split('/')))
        if len(label) == 1:
            label = label[0]
    
    return label


def parse_set1(path):
    
    wb = load_workbook(filename = path)

    sheet_titles = []
    omics_data = defaultdict(lambda:defaultdict(lambda:set()))
    omics_labels = defaultdict(lambda:set())

    for sheet in wb:
        cell_value = sheet['A3'].value

    #     if "Expression data (FC) of the differentially expressed" in sheet['A1'].value:
    #         sheet_title = sheet['A1'].value.split("Expression data (FC) of the differentially expressed ",1)[1]
    #         sheet_title = sheet_title.split(" of HepG2 cells after treatment with ")
    #         sheet_title[1] = sheet_title[1].replace(". Statistical significance (p value < 0.05) is indicated.", "").replace(" CsA for", "")
    #         sheet_titles.append(sheet_title)

        if cell_value and ("Significant " in cell_value or "Metabolite" == cell_value):
            if  cell_value == "Metabolite":
                sheet_title = ("Metabolite", '3 µM', ' 24h or 72h')
                min_row = 3

            else:
                sheet_title = cell_value.split("Significant ",1)[1]
                sheet_title = sheet_title.split(" CsA ")
                sheet_title.append(sheet_title[1].split(" ")[0] + ' h')
                sheet_title[1] = sheet_title[0].split(" ")[1]+ ' µM'
                sheet_title[0] = sheet_title[0].split(" ")[0]
                min_row = 4

            for col in sheet.iter_cols(min_row=min_row):
                col_label = col[0].value
                sheet_omic = sheet_title[0]

                if col_label in ['MicroRNA', 'hgnc symbol', 'Metabolite']:
                    omics_labels[sheet_omic.lower()].update(munge_labels(cell.value) for cell in col[1:])

            sheet_titles.append(sheet_title)

    return omics_labels
    
dataset1_omics_labels = parse_set1(os.path.join(dir_path, 'validation', 'set1.xlsx'))
dataset1_omics_labels

defaultdict(<function __main__.parse_set1.<locals>.<lambda>()>,
            {'genes': {'',
              'tmem64',
              'nfxl1',
              'map3k11',
              'ankrd26',
              'znf581',
              'ints12',
              'tpi1',
              'icam1',
              'hey1',
              'slc9a6',
              'eid1',
              'znf57',
              'btd',
              'mvp',
              'hells',
              'larp1b',
              'pepd',
              'fer',
              'mrps30',
              'spata2l',
              'pgm1',
              'fuca1',
              'fam113a',
              'lbr',
              'pigx',
              'cep89',
              'epo',
              'blvra',
              'c2orf74',
              'c6orf154',
              'clic1',
              'tars',
              'bend3',
              'nfs1',
              'slc29a4',
              'hsf1',
              'mgst1',
              'polq',
              'snapin',
          

In [5]:
print(f'Total number of genes: ({len(dataset1_omics_labels["genes"])})')

print(f'Total number of metabolites: ({len(dataset1_omics_labels["metabolite"])})')

print(f'Total number of miRNAs: ({len(dataset1_omics_labels["micrornas"])})')

total_entities = len(dataset1_omics_labels["metabolite"]) + len(dataset1_omics_labels["micrornas"]) + len(dataset1_omics_labels["genes"])

print(total_entities)

Total number of genes: (4942)
Total number of metabolites: (21)
Total number of miRNAs: (100)
5063


## PathMe Imports

### Graph Universe

In [6]:
# Modify Python Typing arguments if you are using Python < 3.6
# PyBEL union method cannot be used because empty nodes are discarded
def get_nodes_in_database(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return list({
        node
        for network in database_networks
        for node in network.nodes()
    })

def get_edges_in_database(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return list({
        edge
        for network in database_networks
        for edge in network.edges()
    })


kegg_nodes = get_nodes_in_database(KEGG_BEL)
reactome_nodes = get_nodes_in_database(REACTOME_BEL)
wikipathways_nodes = get_nodes_in_database(WIKIPATHWAYS_BEL)


kegg_edges = get_edges_in_database(KEGG_BEL)
reactome_edges = get_edges_in_database(REACTOME_BEL)
wikipathways_edges = get_edges_in_database(WIKIPATHWAYS_BEL)

G = nx.Graph()

G.add_nodes_from(list(set(kegg_nodes + reactome_nodes + wikipathways_nodes)))
G.add_edges_from(list(set(kegg_edges + reactome_edges + wikipathways_edges)))

In [7]:
len(G)

35686

#### Manual Curation
Since some of the entities were not categorized with their corresponding BEL function (e.g., abundance, gene) due to abnormalities in harmonization, we have manually curated them and assigned them to the right modality.

This data can be used by the original database to correct the node label in their corresponding pathway files.

In [8]:
# Entities in WikiPathways that required manual curation
WIKIPATHWAYS_BIOL_PROCESS = {'lipid biosynthesis', 'hsc survival', 'glycolysis & gluconeogenesis', 'triacylglyceride  synthesis', 'wnt canonical signaling', 'regulation of actin skeleton', 'fatty acid metabolism', 'mrna processing major splicing pathway', 'senescence', 'monocyte differentiation', 'pentose phosphate pathway', 'ethanolamine  phosphate', 'hsc differentiation', 'actin, stress fibers and adhesion', 'regulation of actin cytoskeleton', 's-phase progression', 'g1-s transition', 'toll-like receptor signaling pathway', 'regulation of  actin cytoskeleton', 'proteasome degradation', 'apoptosis', 'bmp pathway', 'ampk activation', 'g1/s checkpoint arrest', 'mapk signaling pathway', 'chromatin remodeling and  epigenetic modifications', 'wnt signaling pathway', 'ros production', 'erbb signaling pathway', 'shh pathway', 'inflammation', 'dna replication', 'mrna translation', 'oxidative stress', 'cell cycle checkpoint activation', 'gi/go pathway', 'wnt pathway', 'g1/s transition of mitotic cell cycle', 'modulation of estrogen receptor signalling', 'dna repair', 'bmp canonical signaling', 'igf and insuline signaling', 'unfolded protein response', 'cell death', 'p38/mapk  pathway', 'glycogen metabolism', 'gnrh signal pathway', 'the intra-s-phase checkpoint mediated arrest of cell cycle progression', 'tca cycle', 'mtor protein kinase signaling pathway', 'proteasome  degradation pathway', 'morphine metabolism', 'hsc aging', 'gastric pepsin release', 'parietal cell production', 'prostaglandin pathway', 'cell cycle (g1/s)  progression', 'notch pathway', 'g2/m progression', 'wnt signaling', 'cell adhesion', 'cell cycle progression', 'egfr pathway', 'cell cycle', 'angiogenesis', 'g2/m-phase checkpoint', 'hsc self renewal', '26s proteasome  degradation', 'mapk signaling', 'immune system up or down regulation', 'm-phase progression', 'insulin signaling', 'nf kappa b pathway', 'cell cycle  progression', 'gi pathway', 'cd45+ hematopoietic-    derived cell    proliferation', "kreb's cycle", 'glycogen synthesis', 'apoptosis pathway', 'g1/s progression', 'inflammasome activation', 'melanin biosynthesis', 'proteasomal degradation', 'g2/m checkpoint arrest', 'g1/s cell cycle transition', 'dna damage response', 'gastric histamine release'}
WIKIPATHWAYS_METAB = {'2,8-dihydroxyadenine', '8,11-dihydroxy-delta-9-thc', 'adp-ribosyl', 'cocaethylene', 'dhcer1p', 'ecgonidine', 'f2-isoprostane', 'fumonisins b1', 'iodine', 'l-glutamate', 'lactosylceramide', 'methylecgonidine', 'n-acetyl-l-aspartate', 'nad+', 'nadph oxidase', 'neuromelanin', 'nicotinic acid (na)', 'nmn', 'pip2', 'sphingomyelin', 'thf'}
WIKIPATHWAYS_NAME_NORMALIZATION = {"Ca 2+": "ca 2+", "acetyl coa": "acetyl-coa", "acetyl-coa(mit)": "acetyl-coa", "h20": "h2o"}

# Entities in Reactome that required manual curation
BLACK_LIST_REACTOME = {"5'"}
REACTOME_PROT = {'phospho-g2/m transition proteins', 'integrin alpha5beta1, integrin alphavbeta3, cd47', 'food proteins', 'activated fgfr2', 'adherens junction-associated proteins', 'pi3k mutants,activator:pi3k', 'prolyl 3-hydroxylases', 'gpi-anchored proteins', 'c3d, c3dg, ic3b', 'c4s/c6s chains', 'activated fgfr1 mutants and fusions', 'activated fgfr3 mutants', 'protein', 'cyclin a2:cdk2 phosphorylated g2/m transition protein', 'c4c, c3f', 'activated raf/ksr1', 'activated fgfr1 mutants', 'g2/m transition proteins', 'lman family receptors', 'cyclin', 'usp12:wdr48:wdr20,usp26', 'proteins with cleaved gpi-anchors', 'activated fgfr2 mutants', 'c4d, ic3b', 'c5b:c6:c7, c8, c9', 'cyclin a1:cdk2 phosphorylated g2/m transition protein', 'genetically or chemically inactive braf', 'il13-downregulated proteins', 'activated fgfr4 mutants', 'rna-binding protein in rnp (ribonucleoprotein) complexes', 'effector proteins', 'usp3, saga complex', 'dephosphorylated "receiver" raf/ksr1'}

#### Methods used in this notebook to process node information and create a set for each modality/database

In [9]:
def process_reactome_multiple_genes(genes):
    """Process a wrong ID with multiple identifiers"""
    gene_list = []
    for counter, gene in enumerate(genes):
        
        # Strip the ' gene' prefix
        gene = gene.strip().strip(' gene').strip(' genes')
        
        # First element is always OK
        if counter == 0:
            gene_list.append(gene)
        
        # If the identifier starts the same than the first one, it is right
        elif gene[:2] == genes[0][:2]:
            gene_list.append(gene)
        
        # If the identifier is longer than 2 it is a valid HGNC symbol
        elif len(gene) > 2:
            gene_list.append(gene)
 
        # If they start different, it might have only a number (e.g., 'ABC1, 2, 3') so it needs to be appended
        elif gene.isdigit():
            gene_list.append(genes[0][:-1] + gene)
        
        # If the have only one letter (e.g., HTR1A,B,D,E,F,HTR5A)
        elif len(gene) == 1:
            gene_list.append(genes[0][:-1] + gene)
            
    return gene_list
    
def munge_reactome_gene(gene):
    """Process Reactome gene"""
    if "," in gene:
        return process_reactome_multiple_genes(gene.split(","))
        
    elif "/" in gene:
        return process_reactome_multiple_genes(gene.split("/"))
    
    return gene

def calculate_database_sets(nodes, database):
    """Calculate node sets for each modality in the database"""
    gene_nodes = set()
    mirna_nodes = set()
    metabolite_nodes = set()
    bp_nodes = set()
    
    for node in nodes:
        
        if isinstance(node, ListAbundance) or isinstance(node, Reaction) or not node.name:
            continue
                
        # Lower case name and strip quotes or white spaces
        name = node.name.lower().strip('"').strip()
        
        # Dealing with Genes/miRNAs
        if isinstance(node, CentralDogma):
            
            ##################
            # miRNA entities #
            ##################
            
            if name.startswith("mir"):
                
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            mirna_nodes.add(name.replace("mir-", "mir"))
                    else:
                        mirna_nodes.add(name.strip(' genes').replace("mir-", "mir"))
                        
                    continue
                
                mirna_nodes.add(name.replace("mir-", "mir"))
                
            ##################
            # Genes entities #
            ##################
            
            else:
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            if name in BLACK_LIST_REACTOME: # Filter entities in black list
                                continue
                            elif name.startswith("("): # remove redundant parentheses
                                name = name.strip("(").strip(")")
                                
                            gene_nodes.add(name)
                    else:
                        gene_nodes.add(name)
                    continue
                    
                # WikiPathways and KEGG do not require any processing of genes
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue
                gene_nodes.add(name)
         
        #######################
        # Metabolite entities #
        #######################
        
        elif isinstance(node, Abundance):
            
            if database == 'wikipathways':
                # Biological processes that are captured as abundance in BEL since they were characterized wrong in WikiPathways
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue

                elif node.namespace in {'WIKIDATA', 'WIKIPATHWAYS', 'REACTOME'} and name not in WIKIPATHWAYS_METAB:
                    bp_nodes.add(name)
                    continue
                    
                # Fix naming in duplicate entity
                if name in WIKIPATHWAYS_NAME_NORMALIZATION:
                    name = WIKIPATHWAYS_NAME_NORMALIZATION[name]
                    
            elif database == 'reactome':
                # Curated proteins that were coded as metabolites
                if name in REACTOME_PROT:
                    gene_nodes.add(name)
                    continue
                
                # Flat multiple identifiers (this is not trivial because most of ChEBI names contain commas, 
                # so a clever way to fix some of the entities is to check that all identifiers contain letters)
                elif "," in name and all(
                    string.isalpha() 
                    for string in name.split(",")
                ):
                    for string in name.split(","):
                        metabolite_nodes.add(name)
                    continue
                    
            metabolite_nodes.add(name)

        #################################
        # Biological Processes entities #
        #################################
        
        elif isinstance(node, BiologicalProcess):
            if name.startswith('title:'):
                name = name[6:] # KEGG normalize
            
            bp_nodes.add(name)
        
    return gene_nodes, mirna_nodes, metabolite_nodes, bp_nodes

In [10]:
kegg_genes, kegg_mirna, kegg_metabolites, kegg_bps = calculate_database_sets(
    kegg_nodes, 'kegg'
)

reactome_genes, reactome_mirna, reactome_metabolites, reactome_bps = calculate_database_sets(
    reactome_nodes, 'reactome'
)
wikipathways_genes, wikipathways_mirna, wikipathways_metabolites, wikipathways_bps = calculate_database_sets(
    wikipathways_nodes, 'wikipathways'
)

####  Entity count for each modality in each database

In [11]:
print(f'Total number of genes: KEGG ({len(kegg_genes)}), Reactome ({len(reactome_genes)}), WikiPathways ({len(wikipathways_genes)})')

print(f'Total number of metabolites: KEGG ({len(kegg_metabolites)}), Reactome ({len(reactome_metabolites)}), WikiPathways ({len(wikipathways_metabolites)})')

print(f'Total number of miRNAs: KEGG ({len(kegg_mirna)}), Reactome ({len(reactome_mirna)}), WikiPathways ({len(wikipathways_mirna)})')

print(f'Total number of Biological Processes: KEGG ({len(kegg_bps)}), Reactome ({len(reactome_bps)}), WikiPathways ({len(wikipathways_bps)})')

Total number of genes: KEGG (2086), Reactome (6328), WikiPathways (3025)
Total number of metabolites: KEGG (572), Reactome (2570), WikiPathways (454)
Total number of miRNAs: KEGG (0), Reactome (11), WikiPathways (68)
Total number of Biological Processes: KEGG (147), Reactome (2101), WikiPathways (102)


## Dataset label mapping to PathMe

In [12]:
def check_substrings(dataset_nodes, db_nodes):
    intersection_close = set()
    for entity in dataset_nodes:
        if isinstance(entity, tuple):
            for subentity in entity:
                for entity_db in db_nodes:
                    if entity_db in subentity or subentity in entity_db:
                        intersection_close.add(entity_db)
                        break
        else:
            for entity_db in db_nodes:
                if entity_db in entity or entity in entity_db:
                    intersection_close.add(entity_db)
                    break
    return intersection_close

### Dataset labels

In [13]:
kegg_all_omics_labels = set.union(kegg_mirna, kegg_metabolites, kegg_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(reactome_mirna, reactome_metabolites, reactome_genes)
 
all_omics_labels_dataset1 = set()
for labels in dataset1_omics_labels.values():
    all_omics_labels_dataset1.update(set(labels))
                                         
all_omics_labels_dataset1

{'',
 'c13orf15',
 'atf6',
 'stmn3',
 'khnyn',
 'tmem64',
 'nfxl1',
 'itgae',
 'tspan1',
 'map3k11',
 'ankrd26',
 'znf581',
 'ppan',
 'cyp19a1',
 'klc1',
 'arg1',
 'ints12',
 'tpi1',
 'cstf2t',
 'mtap',
 'icam1',
 'hey1',
 'midn',
 'slc9a6',
 'cln8',
 'eid1',
 'zxdb',
 'agbl5',
 'znf57',
 'btd',
 'acsf2',
 ('rnomir33', 'mmumir33', 'hsamir33a'),
 'klc4',
 'mvp',
 'hsamir3611',
 'hells',
 'larp1b',
 'rai14',
 'pepd',
 'arl2',
 'spon2',
 'fer',
 'mrps30',
 'spata2l',
 'pgm1',
 'hspa5',
 'fuca1',
 'spire2',
 'klhl36',
 'fam113a',
 'kshvmirk123',
 'kctd18',
 'lbr',
 'pigx',
 'hsamir4285',
 'srsf10',
 'cep89',
 'phlda1',
 'epo',
 'tmeff1',
 'eif2ak2',
 'blvra',
 'bbs2',
 'c2orf74',
 'c6orf154',
 'clic1',
 'tars',
 'lsm4',
 'bend3',
 'pgm2l1',
 'gcdh',
 'sdc1',
 'fnbp1l',
 'lrba',
 'ccr6',
 'rsl1d1',
 'nfs1',
 'prox1',
 'slc29a4',
 'hsf1',
 'mgst1',
 'slc27a2',
 'h1f0',
 'polq',
 'asah1',
 'snapin',
 ('hsamir3313p', 'rnomir331', 'mmumir3313p'),
 'dcp2',
 'idi1',
 'pias3',
 'nr1h4',
 'rsrc2',


### KEGG

In [14]:
kegg_genes_intersect = check_substrings(dataset1_omics_labels['genes'], kegg_genes)
print(len(kegg_genes_intersect))

kegg_genes_intersect = kegg_genes.intersection(dataset1_omics_labels['genes'])
print(len(kegg_genes_intersect))

760
689


In [15]:
kegg_metabolites_intersect = kegg_metabolites.intersection(dataset1_omics_labels['metabolite'])
print(len(kegg_metabolites_intersect))

kegg_metabolites_intersect = check_substrings(dataset1_omics_labels['metabolite'], kegg_metabolites)
print(len(kegg_metabolites_intersect))

3
14


In [16]:
kegg_mirna_intersect = kegg_mirna.intersection(dataset1_omics_labels['micrornas'])
print(len(kegg_mirna_intersect))

kegg_mirna_intersect = check_substrings(dataset1_omics_labels['micrornas'], kegg_mirna)
print(len(kegg_mirna_intersect))

0
0


In [17]:
kegg_mapping = kegg_genes_intersect.union(kegg_mirna_intersect, kegg_metabolites_intersect)
len(kegg_mapping)

703

In [18]:
print(len(kegg_mapping)/total_entities)

0.13885048390282442



### Reactome

In [ ]:
reactome_genes_intersect = check_substrings(dataset1_omics_labels['genes'], reactome_genes)
print(len(reactome_genes_intersect))

reactome_genes_intersect = reactome_genes.intersection(dataset1_omics_labels['genes'])
print(len(reactome_genes_intersect))

853
1387


In [ ]:
reactome_metabolites_intersect = reactome_metabolites.intersection(dataset1_omics_labels['metabolite'])
print(len(reactome_metabolites_intersect))

reactome_metabolites_intersect = check_substrings(dataset1_omics_labels['metabolite'], reactome_metabolites)
print(len(reactome_metabolites_intersect))

reactome_metabolites_intersect

2
16


{'acetylcholine',
 'ade',
 'adp-ribose',
 'car',
 'collagen type i fibril with hydroxylysino-5-ketonorleucine crosslinks',
 'damp, dgmp',
 'glu',
 'gly',
 'la',
 'lac',
 'r-nadphx',
 's',
 'ser',
 'tam',
 'tyr',
 'tyrosine kinase inhibitors of fgfr2 mutants'}

In [ ]:
reactome_metabolites_intersect = reactome_metabolites.intersection(dataset1_omics_labels['metabolite'])
print(len(reactome_metabolites_intersect))

reactome_metabolites_intersect = check_substrings(dataset1_omics_labels['micrornas'], reactome_mirna)
print(len(reactome_metabolites_intersect))

print(len(reactome_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], reactome_mirna))

2
4
0


4

In [ ]:
len(reactome_all_omics_labels.intersection(all_omics_labels_dataset1))

1393

### WikiPathways

In [ ]:
len(wikipathways_genes.intersection(dataset1_omics_labels['genes']))

934

In [ ]:
len(wikipathways_metabolites.intersection(dataset1_omics_labels['metabolite']))

11

In [ ]:
print(len(wikipathways_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], wikipathways_mirna))

0


11

In [ ]:
len(wikipathways_all_omics_labels.intersection(all_omics_labels_dataset1))

945

### All db mapping

In [ ]:
all_omics_labels_db = set.union(kegg_all_omics_labels, reactome_all_omics_labels)

In [ ]:
total_entities_db = len(all_omics_labels_db.intersection(all_omics_labels_dataset1))
print(total_entities_db)

print(total_entities_db/total_entities)

1752
0.3460398972940944


## Score Diffusion with diffuPy: Dataset as input + PathMe as background graph

In [ ]:
from diffuPy.diffuse import diffuse

from diffuPy.matrix import Matrix, LaplacianMatrix

### Input  Matrix: Dataset

In [ ]:
kegg_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)

all_omics_labels = set.union(kegg_all_omics_labels, wikipathways_all_omics_labels, reactome_all_omics_labels) 

input_mat = Matrix(rows_labels=all_omics_labels, cols_labels=['Dataset 1'], init=1)

In [ ]:
print(len(input_mat.mat))

3542


### Kernel Matrix: PathMe Universe Graph

#### Graph Universe

In [ ]:
def get_db_graph_universe(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return pybel.struct.union(database_networks)

def get_pathme_graph_universe():
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    if 'pathme_universe_bel_graph.bel.pickle' in os.listdir(PATHME_DIR):
        return pybel.from_pickle(os.path.join(PATHME_DIR, 'pathme_universe_bel_graph.bel.pickle'))

    else:
        pathme_universe = pathme.cli.export_harmonized_universe()
        pybel.to_pickle(pathme_universe, PATHME_DIR)
        pybel.to_graphml(pathme_graph_universe, os.path.join(PATHME_DIR, 'pathme_universe.gml'))
        return pathme_universe

In [ ]:
import pybel_tools

In [ ]:
from pathme.cli import export_harmonized_universe

In [ ]:
pathme_graph_universe_explode = pybel.from_pickle(os.path.join(PATHME_DIR, 'pathme_universe_bel_graph_explode.bel.pickle'))
pathme_graph_universe_explode

In [ ]:
pathme_graph_universe_no_explode = pybel.from_pickle(os.path.join(PATHME_DIR, 'pathme_universe_bel_graph_no_explode.bel.pickle'))
pathme_graph_universe_no_explode

In [ ]:
pathme_graph_universe_no_explode.summarize()
pybel.to_graphml(pathme_graph_universe_no_explode, os.path.join(PATHME_DIR, 'pathme_graph_universe_no_explode.gml'))

PathMe Universe v1.0.0
Number of Nodes: 44195
Number of Edges: 98103
Network Density: 5.02E-05
Number of Components: 5558
Number of Warnings: 0


In [ ]:
pathme_graph_universe_explode.summarize()
pybel.to_graphml(pathme_graph_universe_explode, os.path.join(PATHME_DIR, 'pathme_graph_universe_explode.gml'))

PathMe Universe v1.0.0
Number of Nodes: 21105
Number of Edges: 84953
Network Density: 1.91E-04
Number of Components: 12060
Number of Warnings: 0


In [ ]:
background_mat = LaplacianMatrix(pathme_graph_universe_explode)

/Users/jmarinllao/Documents/CREB/diffuPy/src/diffuPy/miscellaneous.py:19: UserWarning: Graph must be undirected, so it is converted to undirected.
  warnings.warn('Graph must be undirected, so it is converted to undirected.')


In [ ]:
print(background_mat)

In [ ]:
background_mat.rows_labels

In [ ]:
input_mat = input_mat.match_missing_rows(background_mat.rows_labels, 0)

In [ ]:
print(len(background_mat.mat))
print(len(background_mat.rows_labels))

In [ ]:
print(len(input_mat.mat))
print(len(input_mat.rows_labels))

print(len(set(background_mat.rows_labels)))

In [ ]:
set_no = set(background_mat.rows_labels) - set(input_mat.rows_labels).intersection(set(background_mat.rows_labels))

In [ ]:
set_no

In [ ]:
len(set_no)

In [ ]:
set(input_mat.rows_labels).intersection(set(background_mat.rows_labels))

### Compute diffusion scores

In [ ]:
diffuse(input_mat, 'ml', K = background_mat)